## 의사록 섹션 분리(섹션 2,3 만 가져오기)

In [1]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [2]:
import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

In [3]:
import fitz

In [4]:
contents=[]
dates=[]

In [5]:
import os
os.chdir(r'D:\JO\asiaeconomy\kdigital\hong\project\minutes')
from glob import glob
pdfs = glob('*.pdf')
for pdf in pdfs:
    doc = fitz.open('D:/JO/asiaeconomy/kdigital/hong/project/minutes/'+pdf) #for문 하나씩 추가
    page_count = doc.pageCount
    page=0
    content_str=""
    while(page<page_count):
        p = doc.loadPage(page)
        content_str = content_str+p.getText()
        page +=1
    sections, section_texts = preprocess_minutes(content_str)
    dates.append(pdf[0:8])
    contents.append(section_texts[1]+section_texts[2])  #의사록 섹션 2, 섹션3을 하나로 합쳐서 append

In [6]:
print(len(dates))
print(len(contents))

288
288


In [7]:
print(dates)

['20070105', '20070111', '20070125', '20070208', '20070308', '20070322', '20070412', '20070510', '20070608', '20070621', '20070712', '20070726', '20070809', '20070907', '20070920', '20071011', '20071108', '20071122', '20071207', '20071220', '20080104', '20080110', '20080124', '20080213', '20080307', '20080320', '20080410', '20080508', '20080522', '20080612', '20080626', '20080710', '20080724', '20080807', '20080911', '20080925', '20081009', '20081023', '20081027', '20081107', '20081203', '20081211', '20081224', '20090102', '20090109', '20090122', '20090212', '20090225', '20090312', '20090326', '20090409', '20090423', '20090430', '20090512', '20090521', '20090611', '20090625', '20090709', '20090723', '20090811', '20090910', '20090924', '20091009', '20091112', '20091126', '20091210', '20091224', '20100107', '20100108', '20100121', '20100211', '20100311', '20100325', '20100409', '20100422', '20100512', '20100610', '20100624', '20100709', '20100729', '20100812', '20100831', '20100909', '20

In [8]:
print(contents[1])

['일부 위원은 수출기업의 선물환매도로  촉발된 금리재정거래는 단기외채 증가, 시장금리 하락과 이에 따른 통화신용정책 의 유효성 저하 및 급속한 환율하락 등의 부작용을 유발하므로 금리재정거래 규 모 및 자금운용 행태를 정확히 파악하는 한편 금리재정거래가 지속되지 않도록  대책을 마련해야 한다는 의견을 제시하였음.', '이에 대해 관련부서에서는 최근의 선물환매도는 일방향의 환율전망에 기인 한다는 점에서 매도 주체인 대형 수출기업과 환율전망 기관인 국제투자은행, 외신  등과의 접촉 확대를 통해 원화가 지나치게 고평가되어 있다는 점을 주지시키고  있으며 금리재정거래와 관련된 거주자와 비거주자의 해외차입규모, 파생상품거래  및 유가증권투자 동향을 면밀히 모니터링하는 중이라고 설명하였음.', '다른 일부 위원은 외환시장과 원화금융시장 간 유기적인 관계를 고려하여  지급준비율 인상조치 이후 외화자금 유출입 동향을 점검하여야 한다는 의견을 개 진하였음.', '또 다른 일부 위원은 우리나라 외환보유액이 상당한 규모에 이르고 있음에 도 불구하고 외평기금에서 외화외평채를 발행하는 이유를 물었으며, 이에 대해  관련부서에서는 외환보유액 확보 목적으로는 발행할 필요가 없으나 우리나라 민 간기업의 해외차입시 벤치마크로 활용될 수 있는 기준금리를 제공하는 한편 정부 - 4 - 의 경제정책에 대한 대외 IR 기회로 활용하기 위해 이루어지고 있는 것으로 생각 된다고 답변하였음.', '동 위원은 지난해 주요국 통화가 대부분 달러화에 대해 강세를 보인 데 반 해 엔화는 약세를 보인 이유를 물었으며, 이에 대해 관련부서에서는 내외금리차이 에 따라 일본내 거주자들의 해외증권투자가 지속되고 일본내 비거주자들의 초단 기 엔 캐리 트레이드가 계속된 데 주로 기인한다고 답변하였음.', '일부 위원은 당행의 지급준비율 인상조치는  금리정책의 유효성을 제고하고 급속한 유동성 증가세를 완화하기 위한 목적에서  취해졌는데 현재 금융시장에 기대효과가 나타나고 있는지를 관련부서에 물었음.', '이에 대해 관련부서

In [9]:
print(contents[4])

['일부 위원은 원화의 국제화를 위해서 는 제도 정비와 함께 실제 거래가 뒷받침되어야 하는데 최근 일부 조선업체가 선 박대금의 상당부분을 원화로 결제하는 방식의 계약을 성사시킨 것은 원화의 국제 화에 크게 기여할 것으로 생각하며, 원화결제의 활성화는 원화의 국제화와 기업 의 환위험 헷지 및 환거래비용 절감과 함께 그동안 수출 대기업의 선물환매도로  야기됐던 부작용도 상당부분 해소될 수 있을 것으로 생각된다는 견해를 피력하였 음.', '동 위원은 수출대금의 원화결제가 원화환율에 미치는 영향이 중립적이기  위해서는 외국의 수입업체가 국내은행에 비거주자 자유원계정(free won account) 을 개설하여 원화를 예치해놓고 이를 통해서 결제해야 하므로 자유원계정이 활성 화될 수 있도록 관련 제도의 개선을 검토할 필요가 있다는 의견을 제기하였음.', '다른 일부 위원은 헷지펀드의 파산 등으로 인해 국제유동성의 흐름이 급격 히 변동하는 경우 여타 신흥시장국과 같이 국제금융시장 접근성이 다소 제한적인  우리나라에 미치는 부정적인 영향이 클 것이므로 최근 글로벌 증시 하락 등 국제 금융시장의 불안정성과 관련하여 헷지펀드 동향에 대한 모니터링을 강화할 필요 가 있다는 의견을 표명하였음.', '이에 대해 관련부서에서는 주요국 통화가치의 변동폭, 주요국 통화선물 매 도규모 변동추이, 엔화대출 규모, 주요국 자본수지 동향 등을 면밀히 모니터링한  결과 최근의 국제금융시장 불안에는 엔 캐리트레이드를 적극적으로 취급하는 헷 지펀드의 영향이 컸던 것으로 분석되고, 엔 캐리트레이드 청산이 아직은 본격화되 - 5 - 지 않은 것으로 파악되나 1998년의 롱텀 캐피탈 매니지먼트(Long Term Capital  Management) 사태나 러시아 모라토리엄(moratorium) 등의 사례와 같이 시장불안에  편승하여 엔 캐리트레이드가 본격화될 가능성에 유의하여 모니터링을 강화하고  있다고 설명하였음.', '동 위원은 최근의 글로벌 증시 동향을 볼 때 국제금융불안시 신흥시장국  증시 전체에

## 데이터프레임 만들기

In [10]:
import pandas as pd

In [11]:
for i in range(len(dates)):
    dates[i] = dates[i][0:4]+'.'+dates[i][4:6]+'.'+dates[i][6:8]
print(dates)

['2007.01.05', '2007.01.11', '2007.01.25', '2007.02.08', '2007.03.08', '2007.03.22', '2007.04.12', '2007.05.10', '2007.06.08', '2007.06.21', '2007.07.12', '2007.07.26', '2007.08.09', '2007.09.07', '2007.09.20', '2007.10.11', '2007.11.08', '2007.11.22', '2007.12.07', '2007.12.20', '2008.01.04', '2008.01.10', '2008.01.24', '2008.02.13', '2008.03.07', '2008.03.20', '2008.04.10', '2008.05.08', '2008.05.22', '2008.06.12', '2008.06.26', '2008.07.10', '2008.07.24', '2008.08.07', '2008.09.11', '2008.09.25', '2008.10.09', '2008.10.23', '2008.10.27', '2008.11.07', '2008.12.03', '2008.12.11', '2008.12.24', '2009.01.02', '2009.01.09', '2009.01.22', '2009.02.12', '2009.02.25', '2009.03.12', '2009.03.26', '2009.04.09', '2009.04.23', '2009.04.30', '2009.05.12', '2009.05.21', '2009.06.11', '2009.06.25', '2009.07.09', '2009.07.23', '2009.08.11', '2009.09.10', '2009.09.24', '2009.10.09', '2009.11.12', '2009.11.26', '2009.12.10', '2009.12.24', '2010.01.07', '2010.01.08', '2010.01.21', '2010.02.11', '2010

In [12]:
for i in range(len(contents)):
    contents[i] = "@@@".join(contents[i])

In [13]:
minutes_data = {
    'date':dates,
    'content':contents
}

In [14]:
minutes_data = pd.DataFrame(minutes_data)

In [15]:
type(minutes_data)

pandas.core.frame.DataFrame

In [16]:
minutes_data

,date,content
0,2007.01.05,
1,2007.01.11,"일부 위원은 수출기업의 선물환매도로 촉발된 금리재정거래는 단기외채 증가, 시장금리..."
2,2007.01.25,
3,2007.02.08,일부 위원은 지난달 정부가 발표한 해 외투자 확대방안이 실행될 경우 환율안정과 유동...
4,2007.03.08,일부 위원은 원화의 국제화를 위해서 는 제도 정비와 함께 실제 거래가 뒷받침되어야 ...
...,...,...
283,2020.09.24,
284,2020.10.14,
285,2020.10.29,
286,2020.11.26,


In [17]:
for i in range(0,288):
    if minutes_data.loc[i]['content'] == '':
        minutes_data.drop(i,inplace=True)

In [18]:
minutes_data

,date,content
1,2007.01.11,"일부 위원은 수출기업의 선물환매도로 촉발된 금리재정거래는 단기외채 증가, 시장금리..."
3,2007.02.08,일부 위원은 지난달 정부가 발표한 해 외투자 확대방안이 실행될 경우 환율안정과 유동...
4,2007.03.08,일부 위원은 원화의 국제화를 위해서 는 제도 정비와 함께 실제 거래가 뒷받침되어야 ...
6,2007.04.12,대부분의 위원들은 최근 수출업체의 선물환매도로 통화스왑금리가 국채수익률을 밑도는 ...
7,2007.05.10,일부 위원은 최근 일부에서 제시되고 있는 폐쇄형 뮤추얼펀드를 이용한 외환보유액 운...
...,...,...
255,2019.04.18,일부 위원은 금년 들어 은행을 통한 외화자금 유입과 함께 명목실효환율이 완만한 하락...
259,2019.07.18,일부 위원은 최근의 외환수급 동향을 볼 때 외은지점을 중심으로 금융거래가 확대되고 ...
261,2019.08.30,일부 위원은 최근 미·중 무역분쟁의 심화와 함께 미달러화에 대한 위안화의 환율이 7...
263,2019.10.16,일부 위원은 우리나라가 실물경제보다는 금융부문에서 오는 충격에 더 취약 할 가능성이...


In [19]:
print(minutes_data.loc[1]['content'])

일부 위원은 수출기업의 선물환매도로  촉발된 금리재정거래는 단기외채 증가, 시장금리 하락과 이에 따른 통화신용정책 의 유효성 저하 및 급속한 환율하락 등의 부작용을 유발하므로 금리재정거래 규 모 및 자금운용 행태를 정확히 파악하는 한편 금리재정거래가 지속되지 않도록  대책을 마련해야 한다는 의견을 제시하였음.@@@이에 대해 관련부서에서는 최근의 선물환매도는 일방향의 환율전망에 기인 한다는 점에서 매도 주체인 대형 수출기업과 환율전망 기관인 국제투자은행, 외신  등과의 접촉 확대를 통해 원화가 지나치게 고평가되어 있다는 점을 주지시키고  있으며 금리재정거래와 관련된 거주자와 비거주자의 해외차입규모, 파생상품거래  및 유가증권투자 동향을 면밀히 모니터링하는 중이라고 설명하였음.@@@다른 일부 위원은 외환시장과 원화금융시장 간 유기적인 관계를 고려하여  지급준비율 인상조치 이후 외화자금 유출입 동향을 점검하여야 한다는 의견을 개 진하였음.@@@또 다른 일부 위원은 우리나라 외환보유액이 상당한 규모에 이르고 있음에 도 불구하고 외평기금에서 외화외평채를 발행하는 이유를 물었으며, 이에 대해  관련부서에서는 외환보유액 확보 목적으로는 발행할 필요가 없으나 우리나라 민 간기업의 해외차입시 벤치마크로 활용될 수 있는 기준금리를 제공하는 한편 정부 - 4 - 의 경제정책에 대한 대외 IR 기회로 활용하기 위해 이루어지고 있는 것으로 생각 된다고 답변하였음.@@@동 위원은 지난해 주요국 통화가 대부분 달러화에 대해 강세를 보인 데 반 해 엔화는 약세를 보인 이유를 물었으며, 이에 대해 관련부서에서는 내외금리차이 에 따라 일본내 거주자들의 해외증권투자가 지속되고 일본내 비거주자들의 초단 기 엔 캐리 트레이드가 계속된 데 주로 기인한다고 답변하였음.@@@일부 위원은 당행의 지급준비율 인상조치는  금리정책의 유효성을 제고하고 급속한 유동성 증가세를 완화하기 위한 목적에서  취해졌는데 현재 금융시장에 기대효과가 나타나고 있는지를 관련부서에 물었음.@@@이에 대해 관련부서에서는 지준율 

In [20]:
def make_ngram(df):
    tokens=mpck.tokenize(df['content'])
    ngrams=mpck.ngramize(tokens)
    ngrams_text=",".join(ngrams)
    return ngrams_text

In [21]:
from ekonlpy.sentiment import MPCK
mpck = MPCK()

In [23]:
minutes_data['ngrams']=minutes_data.apply(make_ngram,axis=1)

In [24]:
minutes_data

,date,content,ngrams
1,2007.01.11,"일부 위원은 수출기업의 선물환매도로 촉발된 금리재정거래는 단기외채 증가, 시장금리...","fed/NNG;금리/NNG;인하/NNG;기대/NNG;약화/NNG,총액/NNG;한도/..."
3,2007.02.08,일부 위원은 지난달 정부가 발표한 해 외투자 확대방안이 실행될 경우 환율안정과 유동...,"fed/NNG;금리/NNG;인하/NNG;기대/NNG;약화/NNG,민간/NNG;소비/..."
4,2007.03.08,일부 위원은 원화의 국제화를 위해서 는 제도 정비와 함께 실제 거래가 뒷받침되어야 ...,"가계/NNG;부채/NNG;상환/NNG;부담/NNG;증가/NNG,경상/NNG;수지/N..."
6,2007.04.12,대부분의 위원들은 최근 수출업체의 선물환매도로 통화스왑금리가 국채수익률을 밑도는 ...,"유가/NNG;국제/NNG;원자재/NNG;가격/NNG;상승/NNG,경상/NNG;수지/..."
7,2007.05.10,일부 위원은 최근 일부에서 제시되고 있는 폐쇄형 뮤추얼펀드를 이용한 외환보유액 운...,"경상/NNG;수지/NNG;서비스/NNG;수지/NNG;적자/NNG,가계/NNG;부채/..."
...,...,...,...
255,2019.04.18,일부 위원은 금년 들어 은행을 통한 외화자금 유입과 함께 명목실효환율이 완만한 하락...,"글로벌/NNG;경제/NNG;여건/NNG;개선/NNG,가계/NNG;대출/NNG;증가/..."
259,2019.07.18,일부 위원은 최근의 외환수급 동향을 볼 때 외은지점을 중심으로 금융거래가 확대되고 ...,"거시/NNG;건전성/NNG;정책/NNG;강화/NNG,경기/NNG;침체/NNG;가능성..."
261,2019.08.30,일부 위원은 최근 미·중 무역분쟁의 심화와 함께 미달러화에 대한 위안화의 환율이 7...,"회사채/NNG;투자/NNG;수요/NNG;위축/NNG,crs/NNG;금리/NNG;하락..."
263,2019.10.16,일부 위원은 우리나라가 실물경제보다는 금융부문에서 오는 충격에 더 취약 할 가능성이...,"외국인/NNG;원화채권/NNG;투자/NNG;증가/NNG,경상/NNG;수지/NNG;흑..."


In [25]:
minutes_data.to_csv("minutes_data_save.csv",encoding='utf-8')